In [3]:
import os
os.chdir('..')

In [4]:
from torchvision import datasets, transforms

In [8]:
train_ds = datasets.CIFAR10('data/cifar10', download=True) 
#             transform=transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

Files already downloaded and verified


In [34]:
type(train_ds[1][1])

int

In [14]:
import tensorflow as tf
import h5py

In [13]:
help(tf.keras.utils.get_file)

Help on function get_file in module tensorflow.python.keras.utils.data_utils:

get_file(fname, origin, untar=False, md5_hash=None, file_hash=None, cache_subdir='datasets', hash_algorithm='auto', extract=False, archive_format='auto', cache_dir=None)
    Downloads a file from a URL if it not already in the cache.
    
    By default the file at the url `origin` is downloaded to the
    cache_dir `~/.keras`, placed in the cache_subdir `datasets`,
    and given the filename `fname`. The final location of a file
    `example.txt` would therefore be `~/.keras/datasets/example.txt`.
    
    Files in tar, tar.gz, tar.bz, and zip formats can also be extracted.
    Passing a hash will verify the file after download. The command line
    programs `shasum` and `sha256sum` can compute the hash.
    
    Arguments:
        fname: Name of the file. If an absolute path `/path/to/file.txt` is
            specified the file will be saved at that location.
        origin: Original URL of the file.
     

In [12]:
path = tf.keras.utils.get_file(
      'fed_cifar100.tar.bz2',
      origin='https://storage.googleapis.com/tff-datasets-public/fed_cifar100.tar.bz2',
      file_hash='e8575e22c038ecef1ce6c7d492d7abee7da13b1e1ba9b70a7fc18531ba7590de',
      hash_algorithm='sha256',
      extract=True,
      archive_format='tar')

In [27]:
dir_path = os.path.dirname(path)
h5 = h5py.File(os.path.join(dir_path, 'fed_cifar100_test.h5'), "r")["examples"]

In [37]:
client_ids = h5.keys()

In [37]:
def get_cifar100fed_h5(split):
    dir_path = os.path.dirname(get_cifar100fed_h5.path)
    return h5py.File(os.path.join(dir_path, f'fed_cifar100_{split}.h5'), "r")["examples"]

get_cifar100fed_h5.path = tf.keras.utils.get_file(
      'fed_cifar100.tar.bz2',
      origin='https://storage.googleapis.com/tff-datasets-public/fed_cifar100.tar.bz2',
      file_hash='e8575e22c038ecef1ce6c7d492d7abee7da13b1e1ba9b70a7fc18531ba7590de',
      hash_algorithm='sha256',
      extract=True,
      archive_format='tar')

In [61]:
from torch.utils.data import Dataset

class CIFAR10Fed(Dataset):
    N_ELEMENTS_PER_CLIENT = 100
    def __init__(self, split, transform=None):
        if isinstance(split, str) and split == "test":
            self.h5 = get_cifar100fed_h5('test')
            self.clients = list(self.h5.keys())
        else:
            self.h5 = get_cifar100fed_h5('train')
            self.clients = split
        
        self.transform = transform
        
    def __len__(self):
        return len(self.clients) * self.N_ELEMENTS_PER_CLIENT
    
    def __getitem__(self, idx):
        client_id = idx // self.N_ELEMENTS_PER_CLIENT
        item_id = idx % self.N_ELEMENTS_PER_CLIENT
        
        client = self.h5[self.clients[client_id]]
        img = client["image"][item_id]
        label = client["label"][item_id]
        
        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [68]:
ds = CIFAR10Fed('test', transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

In [70]:
ds[0]

(tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.9608, 0.9608, 0.9922],
          [1.0000, 1.0000, 1.0000,  ..., 0.9529, 0.9608, 0.9922],
          [1.0000, 1.0000, 1.0000,  ..., 0.9373, 0.9608, 1.0000]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.8196, 1.0000, 0.9843],
          [1.0000, 1.0000, 1.0000,  ..., 0.8275, 1.0000, 0.9843],
          [1.0000, 1.0000, 1.0000,  ..., 0.8353, 1.0000, 0.9843]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ...,

In [28]:
for k in h5.keys():
    assert len(h5["0"]["label"]) == 100

In [44]:
list(h5.keys())

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99']

In [30]:
105 // 100

1

In [23]:
for k, v in h5["examples"]["1"].items():
    print(k)

coarse_label
image
label


In [38]:
h5["1"]["image"][0]

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  1,   1,   1]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  1,   1,   1]],

       ...,

       [[200, 200, 200],
        [201, 201, 201],
        [202, 202, 202],
        ...,
        [195, 195, 195],
        [197, 197, 197],
        [192, 192, 192]],

       [[202, 202, 202],
        [204, 204, 204],
        [207, 207, 207],
        ...,
        [197, 197, 197],
        [198, 198, 198],
        [196, 196, 196]],

       [[210, 210, 210],
        [212, 212, 212],
        [213, 213, 213],
        ...,
        [193, 193, 193],
        [193, 193, 193],
        [193, 193, 193]]